In [4]:
import requests
from bs4 import BeautifulSoup
import mysql.connector

# Connect to the database
db = mysql.connector.connect(
    host="localhost",
    user="***",
    password="***",
    database="ECommerce"
)
cursor = db.cursor()

BASE_URL = "https://books.toscrape.com/"

In [5]:
# Define BASE_URL and Helper Functions
BASE_URL = "https://books.toscrape.com/"

rating_classes = {
    "One": 1,
    "Two": 2,
    "Three": 3,
    "Four": 4,
    "Five": 5
}


In [8]:
# Extract - Retrieve Data from Website
for category_link in category_links:
    while category_link:
        response = requests.get(category_link)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        books = soup.find_all("article", class_="product_pod")
        for book in books:
            title = book.h3.a.attrs['title']
            price = float(book.select_one('.price_color').text[1:])
            rating_class = book.select_one(".star-rating").get("class")[1]
            availability = book.select_one('.instock.availability').text.strip()
            image_url = BASE_URL + book.select_one('img')['src'].lstrip('../')
            genre = soup.h1.text.strip()

            raw_books_data.append((title, genre, price, rating_class, availability, image_url))
        
        # Check for the next page link and update category_link; if not found, set category_link to None
        next_link = soup.select_one('.next a')
        category_link = BASE_URL + next_link['href'] if next_link else None


In [9]:
# Transform - Process Raw Data

books_data = []

for raw_book in raw_books_data:
    title, genre, price, rating_class, availability, image_url = raw_book
    star_rating = rating_classes.get(rating_class, 0)

    books_data.append((title, genre, price, star_rating, availability, image_url))


In [13]:
#Load - Insert Data into the Database
genres = {}  # Stores genre names and their corresponding IDs

for data in books_data:
    title, genre, price, star_rating, availability, image_url = data

    if genre not in genres:
        cursor.execute("SELECT genre_id FROM genres WHERE genre_name = %s", (genre,))
        genre_id = cursor.fetchone()
        if not genre_id:
            cursor.execute("INSERT INTO genres (genre_name) VALUES (%s)", (genre,))
            db.commit()
            genre_id = cursor.lastrowid
        else:
            genre_id = genre_id[0]
        genres[genre] = genre_id

    insert_query = """
        INSERT INTO products (product_name, genre_id, price, star_rating, availability, image_url) 
        VALUES (%s, %s, %s, %s, %s, %s) ON DUPLICATE KEY UPDATE 
        price = VALUES(price), star_rating = VALUES(star_rating), availability = VALUES(availability), image_url = VALUES(image_url);
    """
    try:
        cursor.execute(insert_query, (title, genres[genre], price, star_rating, availability, image_url))
        db.commit() 
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        db.rollback()


In [12]:
cursor.execute("SELECT * FROM products LIMIT 5")  # Fetches the first 5 rows from the products table.
results = cursor.fetchall()

for row in results:
    print(row)

(1, "It's Only the Himalayas", 1, Decimal('45.17'), 2, 'https://books.toscrape.com/media/cache/27/a5/27a53d0bb95bdd88288eaf66c9230d7e.jpg', 'In stock')
(2, 'Full Moon over Noah’s Ark: An Odyssey to Mount Ararat and Beyond', 1, Decimal('49.43'), 4, 'https://books.toscrape.com/media/cache/57/77/57770cac1628f4407636635f4b85e88c.jpg', 'In stock')
(3, 'See America: A Celebration of Our National Parks & Treasured Sites', 1, Decimal('48.87'), 3, 'https://books.toscrape.com/media/cache/9a/7e/9a7e63f12829df4b43b31d110bf3dc2e.jpg', 'In stock')
(4, 'Vagabonding: An Uncommon Guide to the Art of Long-Term World Travel', 1, Decimal('36.94'), 2, 'https://books.toscrape.com/media/cache/d5/bf/d5bf0090470b0b8ea46d9c166f7895aa.jpg', 'In stock')
(5, 'Under the Tuscan Sun', 1, Decimal('37.33'), 3, 'https://books.toscrape.com/media/cache/98/c2/98c2e95c5fd1a4e7cd5f2b63c52826cb.jpg', 'In stock')


In [14]:
cursor.close()
db.close()  